# The Elite Ant System Algorithm For The Travelling Salesman Problem

Algorithm from the Book **Ant Colony Optimization by Marco Dorigo and Thomas Stützle**

In [155]:
#----------------------------------------
# Chia E. Tungom
# chemago99@yahoo.com
# July 30th 2020
# Shenzhen China
#----------------------------------------

In [156]:
# import libraries

import numpy as np
import math
import random

# Roulette Wheel for Probability

In [157]:
# probability and selection using roulette wheel 

# Calculate cumulative sum for roulette wheel 
def PPi(pi):
    """ pi: list with probability values
        return roulette proportions"""
    n = len(pi)
    ans = []
    
    for i in range(n):
        ans.append( sum([i for i in pi[:i+1]]) )
    return ans 

# Roulette Wheel 
def Roulette(ppi, Pop):
    """ ppi: list of roulette proportions
        Pop: list of corresponding item   [item1, ... , itemN] """
    
    n = len(ppi)
    ra = random.random()
    
    for i in range(n):
        if i == 0 :
            if ra > 0 and ra < ppi[i]:
                return Pop[i]
        else:
            if ra > ppi[i-1] and ra < ppi[i]:
                return Pop[i]

# Calculate Route Cost and Rank Ants

In [158]:
# calculate cost of each route
def Cost(Pop, graph):
    """ Pop: List of All Items in the e.g [[item1], ... , [itemN]]
    graph: cost matrix or graph 
    returns a cost vector for corresponding item """
    
    Cvector = []
    
    for i in range(len(Pop)):
        route = Pop[i]
        cost = 0
        
        for j in range(len(route)-1):
            cost += graph[route[j]][route[j+1]]
        cost += graph[route[0]][route[len(route)-1]]   
        Cvector.append(cost)
        
    return Cvector

# Sort Chromosomes from Best to Worst
def Rank(Pop, graph):
    """ returns sorted Ants """
    cost = Cost(Pop, graph)
    ans = [ x for _,x in sorted(zip(cost, Pop))]
    #cost = [ _ for _,x in sorted(zip(cost, Pop))]  # need to use the true cost
    return ans

# Initialize Pheromone 

In [159]:
# AS initialization
def TSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    pher = [[cost for i in range(n)] for j in range(n)]
    
    return pher

# EAS initialization
def EASTSPher(graph, e, m, rho):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    numer = e + m
    den = rho*cost
    pher = [[numer/den for i in range(n)] for j in range(n)]
    
    return pher

# Ant Colony System
def ACSTSPher(graph):
    
    n = len(graph)
    route = [i for i in range(n)]
    random.shuffle(route)
    
    cost = Cost([route], graph)[0]
    
    den = n*cost
    pher = [[1/den for i in range(n)] for j in range(n)]
    
    return pher

# Calculate Transition Probability

In [186]:
# Transition Probability
def TransitionP(connections, alpha, beta):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    total = sum( [ (i[0]**alpha)*(i[1]**beta) for i in connections])    # sum heuristic and pheromone info
    prob = [( (((i**alpha)*(j**beta))/total) , k) for i,j,k in connections ]  # probability of each hob
    
    return prob

def TransitionP_ACS(connections, beta, alpha = 1):
    """ Takes a list of tuples [(pheromone, distance, hob), ... , (pheromone, distance, hob)]
        return trasition probability for each hob [(probability, hob), ... , (probability, hob)]"""
    
    prob = [( (i**alpha)*(j**beta) , k) for i,j,k in connections ]  # probability of each hob
    Choice = max(prob)
    
    return Choice[1]

# pheromone, distance and hob information
def DPH(phermatrix, graph, connections, preshob):
    """ DPH(Distance Pheromone hob)
        takes a pheromone matrix
        graph matrix 
        connections in graph to use 
        present hob in graph
        and return the pheromone, distance and hob [(pheromone, distance, hob), ... , (pheromone, distance, hob)]"""
    
    dph = []
    for i in range(len(connections)):
        if connections[i] > 0:
            dph.append( (phermatrix[preshob][i], 1/graph[preshob][i], i) )
            
    #dph = [(phermatrix[preshob][i], graph[preshob][i], i) for i in range(len(connections)) if connections[i] > 0 ]
    
    return dph

def Disconnect(visited, connections):
    """ Disconnects already visited hobs
    visited: already visited hobs
    connections: connection vector with 0: no connection x: length of connection"""
    
    for i in visited:
        connections[i] = 0
        
    return connections

# Find Path 

In [187]:
def FindPath(phermatrix, graph, takeoff, t0 , eps = 0.1, q0 = 0.5, alpha = 2, beta = 2):
    
    path = []
    path.append(takeoff)
    
    while len(path) != len(graph):
        
        Cons = graph[takeoff].copy()
        tabu = path.copy()
        Cons = Disconnect(tabu, Cons)
        HP = DPH(phermatrix, graph, Cons, takeoff)
        
        if random.random() <= q0:
            takeoff = TransitionP_ACS(HP, beta)
            path.append(takeoff)
        else:
            prob = TransitionP(HP, alpha, beta)
            takeoff = Roulette(PPi([i[0] for i in prob]), [i[1] for i in prob])
            path.append(takeoff)
        
        # evaporate pheromone
        #print(path, path[-1], path[-2])
        U = (1 - eps)*phermatrix[path[-1]][path[-2]] + (eps*t0)
        phermatrix[path[-1]][path[-2]] = U
        phermatrix[path[-2]][path[-1]] = U
 
    return path

# Get Elite Route and Connections

In [188]:
def Elite(Elite, ant, graph):
    
    if Cost([ant], graph)[0] < Cost([Elite], graph)[0]:
        return ant
    else:
        return Elite

def connections(L):
    """L is a list
    one way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
    
    cons.append((L[-1], L[0]))
    
    return cons

def connectionsE(L):
    """L is a list
    Two way connection """
    
    cons = []
    for i in range(1,len(L)):
        cons.append( (L[i-1], L[i]) )
        cons.append( (L[i], L[i-1]) )
    
    cons.append((L[-1], L[0]))
    cons.append((L[0], L[1]))
    
    return cons

# Update Pheromone 

In [189]:
# Ant Colony System. Global Update and Evaporation
def ElitePheromoneACS(Elite, matrix, rho):
    """route : route taken
    matrix: pheromone matrix"""
    
    bstau = 1/Cost([Elite], matrix)[0]
    #tau = 1
    
    for i in range(len(Elite)-1):
        U = (1-rho)*(matrix[Elite[i]][Elite[i+1]]) + (rho*bstau)
        matrix[Elite[i]][Elite[i+1]] = U 
        matrix[Elite[i+1]][Elite[i]] = U 
    
    U = (1-rho)*(matrix[Elite[-1]][Elite[0]]) + (rho*bstau)
    matrix[Elite[-1]][Elite[0]] = U
    matrix[Elite[0]][Elite[-1]] = U

    return matrix

# Ant Colony System. local Update
def AntPheromoneACS(Ant, matrix, eps, t0):
    """route : route taken
    matrix: pheromone matrix"""
    
    bstau = 1/Cost([Ant], matrix)[0]
    #tau = 1
    
    for i in range(len(Ant)-1):
        U = (1-eps)*(matrix[Ant[i]][Ant[i+1]]) + (eps*t0)
        matrix[Ant[i]][Ant[i+1]] = U 
        matrix[Ant[i+1]][Ant[i]] = U 
    
    U = (1-eps)*(matrix[Ant[-1]][Ant[0]]) + (eps*t0)
    matrix[Ant[-1]][Ant[0]] = U
    matrix[Ant[0]][Ant[-1]] = U

    return matrix

# Generate a Toy Symmetric TSP Problem 

In [190]:
# A random graph
def Graph(dim, bounds):
    b = [(i, str(i)) for i in range(dim)]   
    matrix = []
    
    for i,j in b:
        j=[]
        if i == 0:
            j.append(0)
            matrix.append(j)
        else:
            for k in range(i+1):
                if k == i:
                    j.append(0)
                else:
                    j.append(random.choice([random.randint(bounds[0], bounds[1])]))  # route length
            matrix.append(j)
            
    M = matrix        
    for i in range(len(M)):
        
        for j in range(1,len(M)-len(M[i])+1):
            M[i].append(M[i+j][i]) 
            
    return M

# Ant System Algorithm

In [195]:
def ACS(Ngraph, ants, alpha = 1, beta = 3, rho = 0.5, eps = 0.1, q0 = 0.9, runs = 50, Pher = None):
    
    if Pher == None:
        #Pher = TSPher(Ngraph)
        Pher = ACSTSPher(graph)
    t0 = Pher[0][0]   
    starthob = random.choice([i for i in range(len(Ngraph))])

    GOAT = [i for i in range(len(Ngraph))]  # initial best ant
    
    i = 0
    while i < runs:
        Ants = []
        #starthob = random.choice([i for i in range(len(Ngraph))])
        
        for j in range(ants):
            stof = FindPath(Pher, Ngraph, starthob, t0, eps, q0, alpha, beta)
            Ants.append(stof)
            GOAT = Elite(GOAT, stof, Ngraph)
            
        # Rank Ants by sorting and keep top w-1
        #Ranked = Rank(Ants, graph)[:w-1]
        
        #Pheromone update
        
        ElitePheromoneACS(GOAT, Pher, rho)
        
        #Pheromone evaporation
        #EvPheromone(Pher, rho)
        #EvPheromoneNeg(Pher, rho)
        
        if i%int(runs*0.05) == 0:
            C = Cost(Ants, Ngraph)
            E = Cost([GOAT], Ngraph)[0]
            print("Evolution ", i, " Elite Cost = ", E,  " Minimum Cost = ", min(C), "  and Maximum Cost = ", max(C))
            #print("<<<<<<<<  The Best Route IS  >>>>>>>> ", GOAT)
        i += 1 

    C = Cost(Ants, Ngraph)    
    print("Minimum Cost Found = ", min(C), "Maximum Cost Found = ", max(C))
    return np.array(Ants), min(C) #, np.array(Pher)

# Example on the Toy Problem

In [196]:
# Toy Network graph 
D = 10
bounds = (2,10)
random.seed(10)
graph = Graph(D, bounds)

np.array(graph)

array([[ 0,  2,  9,  9,  7,  8,  5, 10,  6,  9],
       [ 2,  0,  5,  4,  5,  6,  8,  8, 10,  6],
       [ 9,  5,  0, 10,  2,  9,  2,  3,  4, 10],
       [ 9,  4, 10,  0,  4,  6,  4, 10,  7,  3],
       [ 7,  5,  2,  4,  0,  4,  6,  5,  3,  9],
       [ 8,  6,  9,  6,  4,  0,  5,  5,  9,  4],
       [ 5,  8,  2,  4,  6,  5,  0,  2,  5,  4],
       [10,  8,  3, 10,  5,  5,  2,  0,  8,  7],
       [ 6, 10,  4,  7,  3,  9,  5,  8,  0,  8],
       [ 9,  6, 10,  3,  9,  4,  4,  7,  8,  0]])

In [197]:
# Find Shortest Path with the Algorithm

ants = 10
alpha = 1
beta = 3
rho = 0.1
eps = 0.1
q0 = 0.9
runs = 200

ACS(graph, ants, alpha, beta, rho, eps, q0, runs)

Evolution  0  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  44
Evolution  10  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  20  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  30  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  40  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  50  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  60  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  70  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  80  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  90  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  100  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  110  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  33
Evolution  120  Elite Cost =  33  Minimum Cost =  33   and Maximum Cost =  

(array([[6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7],
        [6, 7, 5, 9, 3, 1, 0, 8, 4, 2],
        [6, 2, 4, 8, 0, 1, 3, 9, 5, 7]]), 33)

# Soving Real TSP problems 

In [198]:
# att48

import RouteMatrix as RM
path = './Data/att48.txt'
graph = RM.TSRM(path)


ants = 10
alpha = 1
beta = 3
rho = 0.1
eps = 0.1
q0 = 0.5
runs = 200

ACS(graph, ants, alpha, beta, rho, eps, q0, runs)

Evolution  0  Elite Cost =  46182.9235699377  Minimum Cost =  46182.9235699377   and Maximum Cost =  57887.56961170709
Evolution  10  Elite Cost =  46182.9235699377  Minimum Cost =  46182.9235699377   and Maximum Cost =  46182.923569937724
Evolution  20  Elite Cost =  41220.15022749473  Minimum Cost =  41220.15022749473   and Maximum Cost =  48603.23670873572
Evolution  30  Elite Cost =  41220.15022749472  Minimum Cost =  41220.15022749473   and Maximum Cost =  46400.39366064891
Evolution  40  Elite Cost =  41220.15022749472  Minimum Cost =  41220.15022749472   and Maximum Cost =  41220.15022749473
Evolution  50  Elite Cost =  41220.15022749472  Minimum Cost =  41220.15022749473   and Maximum Cost =  41220.15022749473
Evolution  60  Elite Cost =  41220.15022749472  Minimum Cost =  41220.15022749473   and Maximum Cost =  41220.15022749473
Evolution  70  Elite Cost =  41220.15022749472  Minimum Cost =  41220.15022749473   and Maximum Cost =  46114.40358622766
Evolution  80  Elite Cost = 

(array([[47,  4, 41, 25,  3, 44, 34,  9, 23, 31, 38, 24, 13, 22, 10, 39,
         14, 19, 46, 20, 12, 40, 15, 21,  2,  7,  0,  8, 37, 30, 43, 27,
         16, 42, 26, 18, 36,  5, 29, 35, 17,  6, 11, 32, 45, 33, 28,  1],
        [47,  4, 41, 25,  3, 44, 34,  9, 23, 31, 38, 24, 13, 22, 10, 39,
         14, 19, 46, 20, 12, 40, 15, 21,  2,  7,  0,  8, 37, 30, 43, 27,
         16, 42, 26, 18, 36,  5, 29, 35, 17,  6, 11, 32, 45, 33, 28,  1],
        [47,  4, 41, 25,  3, 44, 34,  9, 23, 31, 38, 24, 13, 22, 10, 39,
         14, 19, 46, 20, 12, 40, 15, 21,  2,  7,  0,  8, 37, 30, 43, 27,
         16, 42, 26, 18, 36,  5, 29, 35, 17,  6, 11, 32, 45, 33, 28,  1],
        [47,  4, 41, 25,  3, 44, 34,  9, 23, 31, 38, 24, 13, 22, 10, 39,
         14, 19, 46, 20, 12, 40, 15, 21,  2,  7,  0,  8, 37, 30, 43, 27,
         16, 42, 26, 18, 36,  5, 29, 35, 17,  6, 11, 32, 45, 33, 28,  1],
        [47,  4, 41, 25,  3, 44, 34,  9, 23, 31, 38, 24, 13, 22, 10, 39,
         14, 19, 46, 20, 12, 40, 15, 21,  2,  7

In [209]:
# burma14

import RouteMatrix as RM
path = './Data/burma14.txt'
graph = RM.TSRM(path)


ants = 40
alpha = 1
beta = 3
rho = 0.1
eps = 0.1
q0 = 0.1
runs = 200

ACS(graph, ants, alpha, beta, rho, eps, q0, runs)

Evolution  0  Elite Cost =  32.02628609456141  Minimum Cost =  32.02628609456141   and Maximum Cost =  51.22901585428313
Evolution  10  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  46.2226283838545
Evolution  20  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  44.92759614723644
Evolution  30  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  45.52562624779327
Evolution  40  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  43.290049619566666
Evolution  50  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  44.93646600331163
Evolution  60  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  44.08950468660385
Evolution  70  Elite Cost =  31.208766207101625  Minimum Cost =  31.208766207101625   and Maximum Cost =  45.24916888433136
Evolution  

(array([[ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  3,  2, 13, 11,  5, 12,  6,  9,  0,  7,  1,  8, 10],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  3,  2, 13,  1,  7,  0,  9,  8, 10, 12,  6, 11,  5],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  3,  2, 13,  1, 10,  8,  9,  0,  7, 12,  6, 11,  5],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 4,  3,  2, 13,  1,  7,  0,  9,  8, 10, 12,  6, 11,  5],
        [ 4,  5, 11,  6, 12, 10,  8,  9,  0,  7,  1, 13,  2,  3],
        [ 